In [1]:
import pyspark.sql.functions as f

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
7676,application_1589299642358_2172,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
%%configure
{
    "conf": {
        "spark.app.name": "datavirus_final"
    }
}

A session has already been started. If you intend to recreate the session with new configurations, please include the -f argument.


In [3]:
# Initialization
spark

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
%%info

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
7611,application_1589299642358_2106,pyspark,idle,Link,Link,
7617,application_1589299642358_2112,pyspark,idle,Link,Link,
7624,application_1589299642358_2119,pyspark,idle,Link,Link,
7632,application_1589299642358_2126,pyspark,idle,Link,Link,
7633,application_1589299642358_2127,pyspark,busy,Link,Link,
7635,application_1589299642358_2129,pyspark,idle,Link,Link,
7640,application_1589299642358_2135,pyspark,idle,Link,Link,
7642,application_1589299642358_2138,pyspark,idle,Link,Link,
7644,application_1589299642358_2140,pyspark,idle,Link,Link,
7646,application_1589299642358_2142,pyspark,dead,Link,Link,


In [38]:
sbb = spark.read.orc('/data/sbb/orc/istdaten')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [39]:
sbb.show(1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+-------------------+------------+-------------+--------------+----------+---------+-----------+---------+-------------------+--------------+-------------+-------+-----------------+------------+-----------+------------------+----------------+-----------+------------------+-------------+
|betriebstag|   fahrt_bezeichner|betreiber_id|betreiber_abk|betreiber_name|produkt_id|linien_id|linien_text|umlauf_id|verkehrsmittel_text|zusatzfahrt_tf|faellt_aus_tf|  bpuic|haltestellen_name|ankunftszeit|an_prognose|an_prognose_status|    abfahrtszeit|ab_prognose|ab_prognose_status|durchfahrt_tf|
+-----------+-------------------+------------+-------------+--------------+----------+---------+-----------+---------+-------------------+--------------+-------------+-------+-----------------+------------+-----------+------------------+----------------+-----------+------------------+-------------+
| 03.09.2018|80:06____:17010:000|   80:06____|           DB|   DB Regio AG|       Zug|    17010|    

In [40]:
sbb = (
    sbb
    
    # filter out any trips that we are not interested in
    .where(sbb.zusatzfahrt_tf == 'false')
    .where(sbb.faellt_aus_tf == 'false')
    .where(sbb.durchfahrt_tf == 'false')
    .where(sbb.ankunftszeit.isNotNull())
    .where(sbb.an_prognose.isNotNull())
    .where(sbb.an_prognose_status.isin('REAL', 'PROGNOSE'))
    
    # calculate delay
    .withColumn('ankunftszeit_ts', f.unix_timestamp(f.col('ankunftszeit'), "dd.MM.yyyy HH:mm").cast('long'))
    .withColumn('an_prognose_ts', f.unix_timestamp(f.col('an_prognose'), "dd.MM.yyyy HH:mm:ss").cast('long'))
    .withColumn('delay', f.col('an_prognose_ts') - f.col('ankunftszeit_ts'))
    .where(f.col('delay').isNotNull())
    
    # sometimes there are more multiple REAL and PROGNOSE values for the exactly same trip on the same day
    # and time. There is no way which one is the closest to the actual arrival time of the train so we 
    # just drop all duplicates and keep one "randomly".
    .select(['betriebstag', 'fahrt_bezeichner', 'produkt_id', 'linien_id', 'linien_text', 'bpuic', 'ankunftszeit', 'an_prognose_status', 'delay'])
    .dropDuplicates(['betriebstag', 'fahrt_bezeichner', 'bpuic', 'ankunftszeit', 'an_prognose_status'])
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [41]:
stations = spark.read.csv("../data/zurich_stations_ids.csv")
stations = (
    stations
    .select(stations._c0.alias('stop_id'))
)
stop_ids = [row['stop_id'] for row in stations.collect()]
stop_ids = spark.sparkContext.broadcast(stop_ids)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [42]:
sbb_zurich = sbb.where(sbb.bpuic.isin(stop_ids.value))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [133]:
delays = (
    sbb_zurich
    
    # for each trip we only kept one REAL and one PROGNOSE row
    # now we aggregate them into the same row for each trip
    .groupBy(['ankunftszeit', 'fahrt_bezeichner', 'bpuic'])
    .agg(
        f.first(sbb_zurich.produkt_id).alias('produkt_id'),
        f.first(sbb_zurich.linien_id).alias('linien_id'),
        f.first(sbb_zurich.linien_text).alias('linien_text'),
        f.first(f.when(sbb_zurich.an_prognose_status == 'REAL', sbb_zurich.delay).otherwise(None)).alias('real_delay'),
        f.first(f.when(sbb_zurich.an_prognose_status == 'PROGNOSE', sbb_zurich.delay).otherwise(None)).alias('prognose_delay'),
    )
    
    # if there is REAL delay we select it, otherwise we take the PROGNOSE delay 
    .withColumn('delay', f.when(f.col('real_delay').isNotNull(), f.col('real_delay')).otherwise(f.col('prognose_delay')))
    .withColumn('is_delayed', f.when(f.col('delay') > 0, 1).otherwise(0))
    
    # create extra columns that contain time without date
    .withColumn('ankunftszeit_minute', sbb_zurich.ankunftszeit.substr(12, 5))
    .withColumn('ankunftszeit_hour', sbb_zurich.ankunftszeit.substr(12, 2))
    .cache()
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [134]:
trip_probability = (
    delays
    
    # group identical trips over all days
    .groupBy(['fahrt_bezeichner', 'bpuic', 'ankunftszeit_minute'])
    
    # calculate delay probability and 1/(mean delay) for each trip
    .agg(
        f.first(delays.produkt_id).alias('produkt_id'),
        f.first(delays.linien_id).alias('linien_id'),
        f.first(delays.linien_text).alias('linien_text'),
        f.mean(delays.is_delayed).alias('delay_probability'),
        (1.0 / f.mean(f.when(delays.is_delayed == 1, f.when(delays.delay > 30*60, 30*60).otherwise(delays.delay)))).alias('delay_parameter'),
        f.count(delays.delay).alias('n')
    )
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [135]:
trip_probability.write.format("orc").save("/user/gflueck/trip_probability.orc", mode='overwrite')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
An error occurred while calling o4918.save.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:224)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:154)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:104)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:102)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:122)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:131)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:127)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$1.apply(SparkPlan.scala:155)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOpera

In [136]:
produkt_probability = (
    delays
    .groupBy(['ankunftszeit_hour', 'produkt_id'])
    .agg(
        f.mean(delays.is_delayed).alias('delay_probability'),
        (1.0 / f.mean(f.when(delays.is_delayed == 1, f.when(delays.delay > 30*60, 30*60).otherwise(delays.delay)))).alias('delay_parameter'),
        f.count(delays.delay).alias('n')
    )
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [137]:
produkt_probability.write.format("orc").save("/user/gflueck/produkt_probability.orc", mode='overwrite')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [138]:
line_probability = (
    delays
    .groupBy(['ankunftszeit_hour', 'linien_id'])
    .agg(
        f.mean(delays.is_delayed).alias('delay_probability'),
        (1.0 / f.mean(f.when(delays.is_delayed == 1, f.when(delays.delay > 30*60, 30*60).otherwise(delays.delay)))).alias('delay_parameter'),
        f.count(delays.delay).alias('n')
    )
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [139]:
line_probability.write.format('orc').save('/user/gflueck/line_probability.orc', mode='overwrite')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [140]:
trip_probability = spark.read.orc("/user/gflueck/trip_probability.orc")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
u'Unable to infer schema for ORC. It must be specified manually.;'
Traceback (most recent call last):
  File "/hdata/sdf/hadoop/yarn/local/usercache/ebouille/appcache/application_1589299642358_2172/container_e06_1589299642358_2172_01_000001/pyspark.zip/pyspark/sql/readwriter.py", line 476, in orc
    return self._df(self._jreader.orc(_to_seq(self._spark._sc, path)))
  File "/hdata/sdf/hadoop/yarn/local/usercache/ebouille/appcache/application_1589299642358_2172/container_e06_1589299642358_2172_01_000001/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/hdata/sdf/hadoop/yarn/local/usercache/ebouille/appcache/application_1589299642358_2172/container_e06_1589299642358_2172_01_000001/pyspark.zip/pyspark/sql/utils.py", line 69, in deco
    raise AnalysisException(s.split(': ', 1)[1], stackTrace)
AnalysisException: u'Unable to infer schema for ORC. It must be specified manually.;'



In [141]:
produkt_probability = spark.read.orc("/user/gflueck/produkt_probability.orc")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [91]:
line_probability = spark.read.orc('/user/gflueck/line_probability.orc')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [100]:
line_probability.where(line_probability.n > 1000).count() * 1.0 / line_probability.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.7102976948916947

In [101]:
line_probability.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

8633

In [128]:
full_probability = (
    trip_probability
    .withColumn('ankunftszeit_hour', trip_probability.ankunftszeit_minute.substr(0, 3))
    .join(f.broadcast(line_probability), ['linien_id', 'ankunftszeit_hour'])
    .join(f.broadcast(produkt_probability), ['produkt_id', 'ankunftszeit_hour'])
    .select(
        trip_probability.ankunftszeit_minute.alias('arrival_time_minute'),
        trip_probability.fahrt_bezeichner.alias('trip_id'),
        trip_probability.bpuic.alias('station_id'),
        trip_probability.produkt_id.alias('transport_type'),
        trip_probability.linien_id.alias('line_id'),
        trip_probability.linien_text.alias('line_text'),
        trip_probability.delay_probability.alias('trip_delay_probability'),
        trip_probability.delay_parameter.alias('trip_delay_parameter'),
        trip_probability.n.alias('trip_n'),
        line_probability.delay_probability.alias('line_delay_probability'),
        line_probability.delay_parameter.alias('line_delay_parameter'),
        line_probability.n.alias('line_n'),
        produkt_probability.delay_probability.alias('transport_delay_probability'),
        produkt_probability.delay_parameter.alias('transport_delay_parameter'),
        produkt_probability.n.alias('transport_n')
    )
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [129]:
full_probability.write.format('orc').save('/user/gflueck/full_probability.orc', mode='overwrite')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [130]:
probability = (
    full_probability
    .select(
        full_probability.arrival_time_minute,
        full_probability.trip_id,
        full_probability.station_id,
        full_probability.transport_type,
        full_probability.line_id,
        full_probability.line_text,
        (
            f.when(full_probability.trip_n > 500, full_probability.trip_delay_probability)
            .otherwise(
                f.when(full_probability.line_n > 500, full_probability.line_delay_probability)
                .otherwise(full_probability.transport_delay_probability)
            )
        ).alias('delay_probability'),
        (
            f.when(full_probability.trip_n > 500, full_probability.trip_delay_parameter)
            .otherwise(
                f.when(full_probability.line_n > 500, full_probability.line_delay_parameter)
                .otherwise(full_probability.transport_delay_parameter)
            )
        ).alias('delay_parameter')  
    )
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [131]:
probability.write.format('orc').save('/user/gflueck/probability.orc', mode='overwrite')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [132]:
probability.show(10, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+---------------+----------+--------------+-------+---------+------------------+--------------------+
|arrival_time_minute|trip_id        |station_id|transport_type|line_id|line_text|delay_probability |delay_parameter     |
+-------------------+---------------+----------+--------------+-------+---------+------------------+--------------------+
| 21:42             |85:11:10:002   |8503202   |Zug           |10     |EC       |0.8202508288487688|0.014650408388357685|
| 02:58             |85:11:13714:001|8503000   |Zug           |13714  |SN1      |0.6519774011299435|0.006361772034664491|
| 02:12             |85:11:13783:001|8503000   |Zug           |13783  |SN8      |0.8833798882681564|0.007784998646087192|
| 10:08             |85:11:14037:001|8503006   |Zug           |14037  |S24      |0.8132095748704679|0.015289947865916156|
| 13:56             |85:11:14051:002|8503010   |Zug           |14051  |S24      |0.81328451151898  |0.015783488520259153|
| 15:14             |85: